In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        return self.fc2(x)

def get_action(state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.randint(action_size)
    else:
        with torch.no_grad():
            return torch.argmax(q_network(state)).item()

def calculate_reward(predicted_rating_change, actual_new_rating, old_rating):
    predicted_new_rating = old_rating + predicted_rating_change
    rating_difference = abs(predicted_new_rating - actual_new_rating)
    reward = max(0, 100 - rating_difference)
    return reward

def update_network(state, action, reward, next_state, done):
    with torch.no_grad():
        target_q_values = target_network(next_state)
        max_target_q_value = torch.max(target_q_values).unsqueeze(0)  # Ensure it's a tensor with a dimension
        target_q_value = reward + gamma * max_target_q_value * (1 - done)

    predicted_q_values = q_network(state)
    predicted_q_value = predicted_q_values[action].unsqueeze(0)  # Ensure it's a tensor with a dimension

    loss = nn.functional.mse_loss(predicted_q_value, target_q_value)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


def update_target(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())

def calculate_epsilon(steps_done):
    return epsilon_end + (epsilon_start - epsilon_end) * np.exp(-1. * steps_done / epsilon_decay)


In [2]:
df = pd.read_csv("tourist.csv")

In [3]:
all_tags = '; '.join(df['Problem Tags'])
tags_split = [tag.strip() for sublist in all_tags.split(';') for tag in sublist.split(',') if tag.strip()]
unique_tags = set(tags_split)
print(unique_tags)

{'string suffix structures', 'constructive algorithms', 'matrices', 'two pointers', 'dfs and similar', 'geometry', 'hashing', 'data structures', 'divide and conquer', 'ternary search', 'math', 'expression parsing', 'dsu', 'trees', 'probabilities', 'brute force', '2-sat', 'chinese remainder theorem', 'binary search', 'flows', 'graphs', 'number theory', 'interactive', 'combinatorics', 'bitmasks', 'strings', 'greedy', 'games', 'sortings', 'fft', 'schedules', 'implementation', 'graph matchings', 'shortest paths', 'dp', 'meet-in-the-middle'}


In [4]:
for tag in unique_tags:
    df[tag] = df['Problem Tags'].apply(lambda x: 1 if tag in x else 0)

df.head()

,Unnamed: 0,Old Rating,New Rating,Contest Duration,Contest Start,Problem Tags,constructive algorithms,dp,combinatorics,math,...,matrices,strings,string suffix structures,geometry,implementation,schedules,ternary search,divide and conquer,games,2-sat
0,0,1400,1602,7200,1267117200,"hashing, implementation; dp, math; geometry",0,1,0,1,...,0,0,0,1,1,0,0,0,0,0
1,1,1602,1764,7200,1270741500,"strings; constructive algorithms, graphs, impl...",1,1,0,0,...,0,1,0,1,1,0,0,0,0,0
2,2,1764,1878,7200,1271346300,"implementation; dp, implementation; number the...",1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,1878,1967,7200,1273154400,"implementation, math; geometry, implementation...",0,1,0,1,...,0,0,0,1,1,0,0,0,0,0
4,4,1967,2063,7200,1277391600,"implementation; dp; greedy, hashing, string su...",0,1,0,0,...,0,0,1,0,1,0,0,1,0,0


In [7]:
tag_columns = list(unique_tags)
print(len(tag_columns))

36


In [ ]:
state_size = 3 + 36 # 36 for problem categories + 3 for other features
action_size = 41
hidden_size = 64
learning_rate = 0.001
gamma = 0.99

epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 200

q_network = QNetwork(state_size, action_size, hidden_size)
target_network = QNetwork(state_size, action_size, hidden_size)
optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)

steps_done = 0
update_target(q_network, target_network)

num_epochs = 1000  # Number of times to iterate over the DataFrame

for epoch in tqdm(range(num_epochs)):
    # Reset initial state at the beginning of each epoch
    initial_tags = df.iloc[0][tag_columns].values
    initial_state = torch.tensor([df.iloc[0]['Old Rating'], df.iloc[0]['Contest Start'], df.iloc[0]['Contest Duration']] + list(initial_tags), dtype=torch.float32)

    for index in range(len(df) - 1):
        current_row = df.iloc[index]
        next_row = df.iloc[index + 1]
        state = initial_state if index == 0 else next_state  # Use initial_state for the first iteration, then next_state

        epsilon = calculate_epsilon(steps_done)
        action = get_action(state, epsilon)

        predicted_rating_change = (action - 20) * 10
        reward = calculate_reward(predicted_rating_change, current_row['New Rating'], current_row['Old Rating'])
        reward = torch.tensor([reward], dtype=torch.float32)
        next_tags = df.iloc[index + 1][tag_columns].values
        next_state = torch.tensor([next_row['Old Rating'], next_row['Contest Start'], next_row['Contest Duration']] + list(next_tags), dtype=torch.float32)
        done = 1 if index == len(df) - 2 else 0

        update_network(state, action, reward, next_state, done)

        steps_done += 1

        if steps_done % 10 == 0:
            update_target(q_network, target_network)

 49%|████▉     | 492/1000 [05:00<05:32,  1.53it/s]

In [ ]:
def evaluate_model(model, evaluation_data):
    total_squared_error = 0
    for _, row in evaluation_data.iterrows():
        tags = row[tag_columns].values
        state = torch.tensor([row['Old Rating'], row['Contest Start'], row['Contest Duration']] + list(tags), dtype=torch.float32)
        with torch.no_grad():
            action = torch.argmax(model(state)).item()
        predicted_rating_change = (action - 10) * 10
        actual_rating_change = row['New Rating'] - row['Old Rating']
        total_squared_error += (predicted_rating_change - actual_rating_change) ** 2

    mse = total_squared_error / len(evaluation_data)
    rmse = mse ** 0.5
    return rmse

# Assuming eval_df is your DataFrame containing evaluation data
eval_df = df  # Replace with your evaluation data
rmse = evaluate_model(q_network, eval_df)
print("Root Mean Squared Error:", rmse)


In [2]:
import requests
import pandas as pd
import time
from tqdm import tqdm

def get_contest_problem_tags(contest_id):
    url = f"https://codeforces.com/api/contest.standings?contestId={contest_id}&from=1&count=1"
    response = requests.get(url)
    if response.status_code != 200:
        return "Error fetching problem tags"

    problems = response.json()['result']['problems']
    tags_list = [', '.join(problem['tags']) for problem in problems]
    return '; '.join(tags_list)  # Concatenate tags for all problems

def get_contest_ratings(username):
    user_contest_url = f"https://codeforces.com/api/user.rating?handle={username}"
    contests_response = requests.get(user_contest_url)
    if contests_response.status_code != 200:
        print("Error fetching contest data")
        return

    contest_history = contests_response.json()["result"]
    data = []

    for i, contest in tqdm(enumerate(contest_history)):  # Adjust the range as needed
        old_rating = 1400 if i == 0 else contest["oldRating"]
        new_rating = contest["newRating"]
        contest_id = contest["contestId"]
        contest_tags = get_contest_problem_tags(contest_id)

        # Get contest details
        contest_details_url = f"https://codeforces.com/api/contest.standings?contestId={contest_id}&from=1&count=1"
        contest_details_response = requests.get(contest_details_url)
        if contest_details_response.status_code != 200:
            print(f"Error fetching details for contest {contest_id}")
            continue

        contest_details = contest_details_response.json()["result"]["contest"]
        contest_duration = contest_details["durationSeconds"]
        contest_start_time = contest_details["startTimeSeconds"]

        data.append([old_rating, new_rating, contest_duration, contest_start_time, contest_tags])
        time.sleep(2)

    return pd.DataFrame(data, columns=['Old Rating', 'New Rating', 'Contest Duration', 'Contest Start', 'Problem Tags'])

# Replace 'username' with the handle of the participant
df = get_contest_ratings("tourist")
print(df)


254it [13:08,  3.10s/it]

     Old Rating  New Rating  Contest Duration  Contest Start  \
0          1400        1602              7200     1267117200   
1          1602        1764              7200     1270741500   
2          1764        1878              7200     1271346300   
3          1878        1967              7200     1273154400   
4          1967        2063              7200     1277391600   
..          ...         ...               ...            ...   
249        3727        3817             10800     1693406100   
250        3817        3751              8100     1695047700   
251        3751        3775             10800     1696084500   
252        3775        3557              9000     1698503700   
253        3557        3585              7200     1699367700   

                                          Problem Tags  
0          hashing, implementation; dp, math; geometry  
1    strings; constructive algorithms, graphs, impl...  
2    implementation; dp, implementation; number the...  
3  

In [6]:
df.to_csv("tourist.csv")